In [1]:
import pandas as pd
from cyvcf2 import VCF
import numpy as np
from math import exp
from ast import literal_eval
from tqdm import tqdm

In [2]:
assembly_all = pd.DataFrame(columns = [0,1,2,3,4])

for chrom in range(1,23):       
    assembly = pd.read_csv(f"assembly/homopolymers_assembly_alleles_chr{chrom}.csv", header=None, 
                        sep = "\t", skiprows = 1)
    assembly[4] = assembly.apply(lambda row: literal_eval(row[4]), axis = 1)
    assembly[5] = assembly.apply(lambda row: literal_eval(row[5]), axis = 1)
    assembly = assembly[assembly[4].apply(lambda x: len(x) != 0)]
    assembly = assembly[assembly[5].apply(lambda x: len(x) == 0)]
    assembly_all = pd.concat([assembly_all, assembly])
        
assembly_all[1] = assembly_all[1].astype(int)
assembly_all[2] = assembly_all[2].astype(int)
assembly_all

,0,1,2,3,4,5
49,chr1,262684,262694,Human_STR_90,"[TTTTTTTTTTTTT, TTTTTTTTTTTTT]",[]
50,chr1,267778,267793,Human_STR_91,"[AAAAAAAAAAAAAAAAA, AAAAAAAAAAAAAAAAA]",[]
73,chr1,591734,591751,Human_STR_115,"[AAAAAAAAAAAAAAAAAAAA, AAAAAAAAAAAAAAAAAAAA]",[]
74,chr1,597686,597699,Human_STR_119,"[AAAAAAAAAAAAAA, AAAAAAAAAAAAAA]",[]
75,chr1,598935,598945,Human_STR_120,"[GGGGGGGGGGG, GGGGGGGGGGG]",[]
...,...,...,...,...,...,...
13818,chr22,50781344,50781362,Human_STR_911881,"[AAAAAAAAAAAAAAAAAAA, AAAAAAAAAAAAAAAAAAA]",[]
13819,chr22,50790606,50790627,Human_STR_911886,"[AAAAAAAAAAAAAAAAAAAA, AAAAAAAAAAAAAAAAAAAAAAA...",[]
13820,chr22,50796314,50796328,Human_STR_911887,"[CCCCCCCCCCCCC, CCCCCCCCCCCCCC]",[]
13821,chr22,50804642,50804652,Human_STR_911890,"[TTTTTTTTTTT, TTTTTTTTTTTT]",[]


In [3]:
####### Read LongTR output on HG002 #######
calls = []
for chrom in range(1,23):
    vcf = VCF(f"LongTR_calls/trio_homo_longtr_stutter_chr{chrom}.vcf.gz", samples="HG002")
    for variant in vcf:
        if variant.CHROM == "chrX" or variant.CHROM == "chrY":
            break
        calls.append([variant.CHROM, variant.POS, variant.INFO['START'], variant.INFO['END'],
                      variant.REF, variant.ID, variant.gt_bases[0].split("|"), variant.format('GB')[0]])
LongTR_data_stutter = pd.DataFrame(calls, 
    columns=["chrom","pos_LongTR_stutter", "start_LongTR_stutter",
             "end_LongTR_stutter", "ref_LongTR_stutter", "ID",
             "gbs_LongTR_stutter", 'gb_LongTR_stutter'])

####### Read LongTR without stutter output on HG002 #######
calls = []

vcf = VCF(f"LongTR_calls/trio_homo_longtr_no_stutter.vcf.gz", samples="HG002")
for variant in vcf:
    if variant.CHROM == "chrX" or variant.CHROM == "chrY":
        break
    calls.append([variant.CHROM, variant.POS, variant.INFO['START'], variant.INFO['END'],
                  variant.REF, variant.ID, variant.gt_bases[0].split("|"), variant.format('GB')[0]])
LongTR_data_no_stutter = pd.DataFrame(calls, 
    columns=["chrom","pos_LongTR_no_stutter", "start_LongTR_no_stutter",
             "end_LongTR_no_stutter", "ref_LongTR_no_stutter", "ID",
             "gbs_LongTR_no_stutter", 'gb_LongTR_no_stutter'])


####### Read TRGT output on HG002 #######
calls = []
vcf = VCF("TRGT_calls/HG002_homo_trgt_hg38.sorted.vcf.gz")
for variant in vcf:
    if variant.CHROM == "chrX" or variant.CHROM == "chrY":
        break
    calls.append([variant.CHROM, variant.POS, variant.REF, 
                  variant.INFO['TRID'], variant.gt_bases[0].split("/")])
TRGT_data = pd.DataFrame(calls, 
    columns=["chrom","pos_TRGT", "ref_TRGT", "ID", "gbs_TRGT"])


In [4]:
def fix_seq(pos_hipstr, start_hipstr, ref_hipstr, seq_hipstr, end_hipstr):
    if np.isnan(pos_hipstr):
        return np.nan
    diff_pos = int(start_hipstr - pos_hipstr)
    end_hipstr_new = pos_hipstr + len(ref_hipstr) - 1
    diff_end = int(end_hipstr_new - end_hipstr)
    assert(diff_pos >= 0 and diff_end >= 0) # becase only HipSTR might change the pos
    if diff_pos > 0:
        seq_hipstr = [s[diff_pos:] for s in seq_hipstr]
    if diff_end > 0: 
        seq_hipstr = [s[0:len(s) - diff_end] for s in seq_hipstr]
    alleles_hipstr = sorted(seq_hipstr)
    return alleles_hipstr

both = pd.merge(TRGT_data, LongTR_data_stutter, on = ['chrom', 'ID'])
both = pd.merge(both, LongTR_data_no_stutter, on = ['chrom', 'ID'])

both['gbs_LongTR_stutter_corrected'] = both.apply(lambda row: fix_seq(row['pos_LongTR_stutter'], 
                                                              row['start_LongTR_stutter'], row['ref_LongTR_stutter'],
                                                              row['gbs_LongTR_stutter'], row['end_LongTR_stutter']), axis = 1)

both['gbs_LongTR_no_stutter_corrected'] = both.apply(lambda row: fix_seq(row['pos_LongTR_no_stutter'], 
                                                              row['start_LongTR_no_stutter'], row['ref_LongTR_no_stutter'],
                                                              row['gbs_LongTR_no_stutter'], row['end_LongTR_no_stutter']), axis = 1)



In [5]:
both_assembly = pd.merge(both, assembly_all, left_on = 'ID', right_on=3)
both_assembly['gbs_TRGT'] = both_assembly['gbs_TRGT'].apply(lambda x: sorted(x))
both_assembly[4] = both_assembly[4].apply(lambda x: sorted(x))


In [6]:
print(len(both_assembly[both_assembly['gbs_TRGT'] == both_assembly[4]]) / len(both_assembly))

0.8599225995038957


In [7]:
print(len(both_assembly[both_assembly['gbs_LongTR_stutter_corrected'] == both_assembly[4]]) / len(both_assembly))

0.8495154749529112


In [8]:
print(len(both_assembly[both_assembly['gbs_LongTR_no_stutter_corrected'] == both_assembly[4]]) / len(both_assembly))

0.718886698848256
